# PDFパーサーの使い方 

以下のサイトを参考にしました．  
[【PDFMiner】PDFからテキストの抽出](https://qiita.com/mczkzk/items/894110558fb890c930b5)  
[Programming with PDFMine](https://www.unixuser.org/~euske/python/pdfminer/programming.html)

In [2]:
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTContainer, LTTextBox
from pdfminer.pdfinterp import PDFPageInterpreter, PDFResourceManager
from pdfminer.pdfpage import PDFPage

In [3]:
from pathlib import Path

## LayoutオブジェクトからLTTextBoxのリストを取得する関数

`device`オブジェクトの`get_result`メソッドが返す`layout`オブジェクトのうち，`LTTextBox`(テキストが入っている)のみ取り出し，リストにする．

In [4]:
def find_textboxes_recursively(layout):
    """
    再帰的にテキストボックス（LTTextBox）を探して、テキストボックスのリストを取得する。
    """
    # LTTextBoxを継承するオブジェクトの場合は1要素のリストを返す。
    if isinstance(layout, LTTextBox):
        text_boxes = [layout]
        return text_boxes  # 返すのはリスト

    # LTContainerを継承するオブジェクトは子要素を含むので、再帰的に探す。
    if isinstance(layout, LTContainer):
        text_boxes = []
        for child in layout:
            text_boxes.extend(find_textboxes_recursively(child))  # 再帰的にリストをextend
            
        return text_boxes

    return []  # 何も取得できなかった場合は空リストを返す

## パースに必要なクラスの作成 

In [5]:
# Layout Analysisのパラメーターを設定。縦書きの検出を有効にする。
laparams = LAParams(detect_vertical=True)

# 共有のリソースを管理するリソースマネージャーを作成。
resource_manager = PDFResourceManager(caching=False)  # この引数によってエラーが出なくなる

# ページを集めるPageAggregatorオブジェクトを作成。
device = PDFPageAggregator(resource_manager, laparams=laparams)

# Interpreterオブジェクトを作成。
interpreter = PDFPageInterpreter(resource_manager, device)

## 二段組専用のソート方法 

In [6]:
class Sort2Column():
    def __init__(self, layout_x0, layout_x1):
        self.half_x = (layout_x0 + layout_x1)/2
    
    def __call__(self, text_box):
        if text_box.x0 < self.half_x:
            left_or_right = -1  # it mean left
            
        else:
            left_or_right = 1  # it mean right
            
        return (left_or_right, -text_box.y1)

In [7]:
sorted([0,2,1])

[0, 1, 2]

## ファイルを読み込み，パースを行う

qiitaの記事では，documentオブジェクトは必要ない`PDFPage.get_pages`メソッドを利用する．このメソッドはファイルオブジェクトを引数にとる．

In [32]:
#file_path = Path("./PDFs/IS1-20.pdf")
file_path = Path("/home/umelab-server2020/workspace/paper_parse/paper_example/ロボット学会/rsj_2019.pdf")
#file_path = Path("/home/umelab-server2020/workspace/paper_parse/paper_example/SSII/ssii_2018.pdf")
#file_path = Path("/home/umelab-server2020/workspace/paper_parse/paper_example/robomech/robomech_2019.pdf")
#file_path = Path("/home/umelab-server2020/workspace/paper_parse/paper_example/MiRU/miru_2019.pdf")
#file_path = Path("/home/umelab-server2020/workspace/paper_parse/paper_example/ViEW/view_2017.pdf")
#file_path = Path("/home/umelab-server2020/workspace/paper_parse/paper_example/robotics_symposia/robosym_2020.pdf")

In [33]:
with open(file_path, "rb") as f:
    for page in PDFPage.get_pages(f):
        interpreter.process_page(page)  # ページを処理する。
        layout = device.get_result()  # LTPageオブジェクトを取得。
        text_boxes = find_textboxes_recursively(layout)      

        # text_boxの座標値毎にソート，複数キーのソート
        #text_boxes.sort(key=lambda text_box: (-text_box.y1, text_box.x0))  # y1がy座標，x0がx座標らしい(画像座標なので，y1を負に)
        #text_boxes.sort(key=lambda text_box: (text_box.x0, -text_box.y1)) # 正直二段組のデータでの取得は難しく，ある程度1段踏み前提
        
        # 二段組のソート
        sort2column = Sort2Column(layout_x0=layout.x0, layout_x1=layout.x1)
        text_boxes.sort(key=sort2column)
        for box in text_boxes:
            print("------------------")
            print(box.get_text().strip())  # 末尾の文字を削除
            print("x0:{},x1:{}".format(box.x0, box.x1))
            print("y0:{},y1:{}".format(box.y0, box.y1))
            
        print("-------pages---------")

------------------
多品種ばら積みピッキングにおける
x0:165.118,x1:430.15799999999996
y0:768.1712,y1:788.1982849999999
------------------
物体間の上下関係の予測とデータセットの提案
x0:123.70599999999999,x1:471.57099999999997
y0:743.2642,y1:763.2912849999999
------------------
○稲垣雄介 荒木諒介 平川翼 山下隆義 藤吉弘亘 (中部大学)
x0:146.04199999999997,x1:449.23292799999996
y0:718.02673,y1:732.711725
------------------
1. はじめに
x0:56.692999999999984,x1:126.61499999999998
y0:683.8137300000001,y1:698.659767
------------------
多品種ばら積みのピッキングにおいて，オクルージョ
ンが発生した物体を把持する場合，他物体が上に存在
するため，把持に失敗するという問題がある．これを
解決するには，物体間の上下関係を予測して把持戦略
を決定する必要がある．そこで，本研究では，ばら積み
された物体間の上下関係を獲得するための新たなデー
タセットである ARC Multi-task Dataset を提案し，物
体間の上下関係の予測を行う．
2. 関連データセット
x0:56.692999999999984,x1:288.78923200000014
y0:561.5217300000003,y1:681.82095
------------------
多品種ばら積みのピッキングを対象としたデータベー
スを表 1 に示す．Grasping Dataset [3] は物体の把持可
能な領域に対するセグメンテーションのデータ，Ama-
zon Picking Challenge Object Scans(APCOS) [2] は
RGB-D 点群に対するセグメンテーションデータが含ま
れている．JSK のデータセット [4] はインスタンスセグ
メンテーションラベルと各インスタンスに対する

### documentオブジェクトを利用する場合 